In [1]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
import random

import django
django.setup()

from bookstore.models import *
# from mixer.backend.django import mixer
from mixer.backend.django import Mixer

mixer = Mixer(commit=False)

## Fake data generator

In [2]:
MULTIPLIER = 30
AUTHOR_COUNT = MULTIPLIER * 5
PUBLISHER_COUNT = MULTIPLIER * 5
TAGS_COUNT = MULTIPLIER * 5
SERIES_COUNT = MULTIPLIER * 5
IDENTIFIERS_COUNT = MULTIPLIER * 30
BOOKS_COUNT = MULTIPLIER * 30

def generate_fields():
    print('Generating Authors')
    authors = mixer.cycle(AUTHOR_COUNT).blend(Author, name=mixer.faker.name)
    print('Generating Publishers')
    publishers = mixer.cycle(PUBLISHER_COUNT).blend(Publisher, name=mixer.faker.company)
    print('Generating Tags')
    tags = mixer.cycle(TAGS_COUNT).blend(Tag, name=mixer.faker.word)
    print('Generating Series')
    series = mixer.cycle(SERIES_COUNT).blend(Series, title=mixer.faker.sentence)
    print('Generating Identifiers')
    identifiers = mixer.cycle(IDENTIFIERS_COUNT).blend(Identifier, name='isbn13', value=mixer.faker.isbn13)
    
    Author.objects.bulk_create(authors)
    Publisher.objects.bulk_create(publishers)
    Tag.objects.bulk_create(tags)
    Series.objects.bulk_create(series)
#     Identifier.objects.bulk_create(identifiers)
    
    print('Updating models')
    authors = Author.objects.all()[:AUTHOR_COUNT]
    publishers = Publisher.objects.all()[:PUBLISHER_COUNT]
    tags = Tag.objects.all()[:TAGS_COUNT]
    series = Series.objects.all()[:SERIES_COUNT]
#     identifiers = Identifier.objects.all()[:IDENTIFIERS_COUNT]
    
    print('Generating Books')
    books = mixer.cycle(BOOKS_COUNT).blend(Book, title=mixer.faker.sentence, user=mixer.SELECT, publisher=mixer.SELECT, series=mixer.SELECT)
    Book.objects.bulk_create(books)
    books = Book.objects.all()[:BOOKS_COUNT]
    
    for book in books:
        book.authors.add(random.choice(authors))
        book.tags.add(random.choice(tags))
        book.identifiers.add(random.choice(identifiers), bulk=False)
        book.save()
    #[book.save() for book in books]

In [3]:
generate_fields()

Generating Authors
Generating Publishers
Generating Tags
Generating Series
Generating Identifiers
Updating models
Generating Books


## Database Queries

In [ ]:
# вывод списка книг по id
[book.id for book in Book.objects.all()]


In [ ]:
# вывод списка книг по названию
[book.title for book in Book.objects.order_by('title')]


In [ ]:
# вывод списка книг по дате публикации
[book.published_at for book in Book.objects.order_by('published_at')]


In [ ]:
# вывод списка книг по рейтингу
[book.rating for book in Book.objects.order_by('rating')]


In [ ]:
# вывод списка книг по рандому
import random
books = [book for book in Book.objects.all()]
random.shuffle(books)


In [ ]:
# вывод списка книг по автору

In [21]:
import random
books = Author.objects.filter(pk=35).first().books.all()
books

<QuerySet [<Book: Book object (42)>, <Book: Book object (43)>]>

In [ ]:
objs_count = Book.objects.count()
random_ids = [i for i in range(1, objs_count+1)]
random.shuffle(random_ids)

In [ ]:
random_ids

In [7]:
authors = [[book.title for book in author.books.all()] for author in Author.objects.all()]

NameError: name 'author' is not defined

In [15]:
with mixer.ctx(commit=False):
    series = mixer.cycle(3).blend(Series, title=mixer.faker.sentence)
    books = mixer.cycle(30).blend(Book, user=mixer.SELECT)
    publishers = mixer.cycle(5).blend(Publisher, name=mixer.faker.company)
    [setattr(book, 'series', (random.choice(series))) for book in books]

In [9]:
series

[<Series: Democrat ten state stock up after.>,
 <Series: It even mind late left response.>,
 <Series: Specific response home question already myself very.>]

In [12]:
books[1].series

<Series: It even mind late left response.>

In [18]:
publishers

[<Publisher: Tate, Allen and Medina>,
 <Publisher: Gross LLC>,
 <Publisher: Barker-Benton>,
 <Publisher: Flynn, Camacho and Spears>,
 <Publisher: Forbes, Cisneros and Vaughan>]

In [4]:
authors = mixer.cycle(5*MULTIPLIER).blend(Author, name=mixer.faker.name)

In [6]:
authors[2].pk

5363